---
## Quick Start

Create a wakefield, evaluate it, and apply it to particles in just a few lines:

In [ ]:
from pmd_beamphysics.wakefields import ResistiveWallWakefield
from pmd_beamphysics import ParticleGroup
import numpy as np

# Create wakefield from material preset
wake = ResistiveWallWakefield.from_material(
    "copper-slac-pub-10707",
    radius=2.5e-3,  # 2.5 mm pipe radius
    geometry="flat",
)


wake.plot(zmax=200e-6, normalized=True)

In [ ]:
# Load particles and apply wakefield over 10 m
P = ParticleGroup("../data/bmad_particles2.h5")

P_after = wake.apply_to_particles(P, length=1.0)  # Returns modified copy

print(f"Mean energy change: {(P_after['mean_energy']- P['mean_energy'])*1e-6:.3f} MeV")

In [ ]:
# Visualize the wakefield and its effect
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Left: Wake function
ax = axes[0]
z = np.linspace(0, 200e-6, 200)
W = wake.wake(-z)  # Negative z = behind source
ax.plot(z * 1e6, W * 1e-12)
ax.set_xlabel(r"Distance behind source $|z|$ (µm)")
ax.set_ylabel(r"$W(z)$ (V/pC/m)")
ax.set_title("Wakefield")

# Right: Per-particle kicks
ax = axes[1]
kicks = wake.particle_kicks(P)
ax.scatter((P.z - P.z.mean()) * 1e6, kicks * 1e-6, s=1, alpha=0.5)
ax.set_xlabel(r"$z - \langle z \rangle$ (µm)")
ax.set_ylabel("Kick (MeV/m)")
ax.set_title("Particle Kicks")

plt.tight_layout()

---
## Setup

In [ ]:
from pmd_beamphysics.wakefields import (
    ResistiveWallWakefield,  # Accurate, impedance-based
    ResistiveWallPseudomode,  # Fast, pseudomode approximation
)
from pmd_beamphysics import ParticleGroup
from pmd_beamphysics.units import epsilon_0

import numpy as np
import matplotlib.pyplot as plt

---
## Creating Wakefield Models

### Direct Construction

Specify the pipe geometry and material properties directly:

In [ ]:
# Accurate impedance-based model
wake = ResistiveWallWakefield(
    radius=2.5e-3,  # Pipe radius [m] (or half-gap for flat)
    conductivity=6.5e7,  # DC conductivity σ₀ [S/m]
    relaxation_time=27e-15,  # Drude relaxation time τ [s]
    geometry="round",  # "round" or "flat"
)
wake

In [ ]:
# Fast pseudomode approximation
wake_fast = ResistiveWallPseudomode(
    radius=2.5e-3,
    conductivity=6.5e7,
    relaxation_time=27e-15,
    geometry="round",
)
wake_fast

### Material Presets

Common materials with known conductivity and relaxation times are available:

In [ ]:
# List available materials
list(ResistiveWallWakefield.MATERIALS)

In [ ]:
# Create from material preset
wake_cu = ResistiveWallWakefield.from_material(
    "copper-slac-pub-10707",
    radius=4.5e-3,
    geometry="flat",
)
wake_cu

In [ ]:
# Material properties
print(f"Conductivity: {wake_cu.conductivity:.2e} S/m")
print(f"Relaxation time: {wake_cu.relaxation_time:.2e} s")
print(f"Characteristic length s₀: {wake_cu.s0*1e6:.2f} µm")

---
## Evaluating the Wakefield

### Wake Function W(z)

The wakefield is defined for a trailing particle at position $z$ relative to the source:
- $z < 0$: Behind the source (trailing particle feels the wake)
- $z > 0$: Ahead of the source (causality requires $W = 0$)

In [ ]:
# Evaluate at single point
z = -10e-6  # 10 µm behind source
W = wake.wake(z)
print(f"W({z*1e6:.0f} µm) = {W:.3e} V/C/m = {W*1e-12:.3f} V/pC/m")

In [ ]:
# Evaluate on array
z_arr = np.linspace(-200e-6, 10e-6, 500)
W_arr = wake.wake(z_arr)

plt.figure(figsize=(8, 4))
plt.plot(-z_arr * 1e6, W_arr * 1e-12)
plt.axvline(0, color="r", ls="--", label="Source position")
plt.xlabel(r"Distance behind source $|z|$ (µm)")
plt.ylabel(r"$W(z)$ (V/pC/m)")
plt.title("Resistive Wall Wakefield")
plt.legend()
plt.xlim(-20, 200)

### Impedance Z(k)

The longitudinal impedance as a function of wavenumber $k$:

In [ ]:
k = np.linspace(0, 5e5, 500)
Z = wake.impedance(k)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

ax = axes[0]
ax.plot(k * 1e-3, np.real(Z))
ax.set_xlabel(r"$k$ (1/mm)")
ax.set_ylabel(r"Re[$Z(k)$] (Ω/m)")
ax.set_title("Real Part (Resistive)")

ax = axes[1]
ax.plot(k * 1e-3, np.imag(Z))
ax.set_xlabel(r"$k$ (1/mm)")
ax.set_ylabel(r"Im[$Z(k)$] (Ω/m)")
ax.set_title("Imaginary Part (Reactive)")

plt.tight_layout()

### Built-in Plot Method

In [ ]:
wake.plot()

---
## Round vs Flat Geometry

The flat (parallel plate) geometry has a higher wake amplitude due to the different boundary conditions:

In [ ]:
params = dict(radius=2.5e-3, conductivity=6.5e7, relaxation_time=27e-15)

wake_round = ResistiveWallWakefield(**params, geometry="round")
wake_flat = ResistiveWallWakefield(**params, geometry="flat")

z = np.linspace(-200e-6, 0, 300)

plt.figure(figsize=(8, 4))
plt.plot(-z * 1e6, wake_round.wake(z) * 1e-12, label="Round")
plt.plot(-z * 1e6, wake_flat.wake(z) * 1e-12, "--", label="Flat")
plt.xlabel(r"Distance behind source $|z|$ (µm)")
plt.ylabel(r"$W(z)$ (V/pC/m)")
plt.title("Round vs Flat Geometry")
plt.legend()

---
## Convolution with Charge Density

The `convolve_density` method computes the integrated wake potential from a charge density distribution:

$$V(z) = \int_{z}^{\infty} \rho(z') \, W(z - z') \, dz'$$

In [ ]:
# Create a Gaussian bunch density
n_bins = 500
dz = 1e-6  # 1 µm spacing
z = np.arange(n_bins) * dz

Q_total = 100e-12  # 100 pC
sigma_z = 10e-6  # 10 µm RMS
z0 = z[n_bins // 2]  # Center

density = (
    Q_total / (sigma_z * np.sqrt(2 * np.pi)) * np.exp(-0.5 * ((z - z0) / sigma_z) ** 2)
)

print(f"Total charge: {np.sum(density) * dz * 1e12:.1f} pC")

In [ ]:
# Compute integrated wake
V = wake.convolve_density(density, dz)

fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

# Density
ax = axes[0]
ax.fill_between((z - z0) * 1e6, density * 1e3, alpha=0.5)
ax.set_ylabel(r"Density (mC/m)")
ax.set_title("Charge Density")

# Wake potential
ax = axes[1]
ax.plot((z - z0) * 1e6, V * 1e-6)
ax.set_xlabel(r"$z - z_0$ (µm)")
ax.set_ylabel(r"Wake potential (MV/m)")
ax.set_title("Integrated Wake (head loses energy, tail gains)")
ax.axhline(0, color="k", lw=0.5)

plt.tight_layout()

---
## Particle Methods

### Computing Per-Particle Kicks

The `particle_kicks` method computes the wakefield-induced momentum change for each particle:

In [ ]:
# Load particles
P = ParticleGroup("../data/bmad_particles2.h5")
P.drift_to_t()  # Align at constant time
P

In [ ]:
# Compute kicks [eV/m]
kicks = wake.particle_kicks(P)

plt.figure(figsize=(8, 4))
plt.scatter((P.z - P.z.mean()) * 1e6, kicks * 1e-6, s=1, alpha=0.5)
plt.xlabel(r"$z - \langle z \rangle$ (µm)")
plt.ylabel("Kick (MeV/m)")
plt.title("Per-Particle Wakefield Kicks")
plt.axhline(0, color="r", ls="--", alpha=0.5)

### Applying to Particles

The `apply_to_particles` method applies the kicks to a `ParticleGroup`:

In [ ]:
# Apply wakefield over 10 m (returns a copy by default)
length = 10.0  # meters
P_after = wake.apply_to_particles(P, length=length)

print(f"Before: mean pz = {P['mean_pz']/1e6:.3f} MeV/c")
print(f"After:  mean pz = {P_after['mean_pz']/1e6:.3f} MeV/c")
print(f"Change: {(P_after['mean_pz'] - P['mean_pz'])/1e6:.3f} MeV/c")

In [ ]:
# Modify in-place
P_copy = P.copy()
wake.apply_to_particles(P_copy, length=10.0, inplace=True)
print(f"In-place modification: mean pz = {P_copy['mean_pz']/1e6:.3f} MeV/c")

### ParticleGroup Wakefield Plot

A convenience method for visualizing the wakefield effect on a particle distribution:

In [ ]:
P.drift_to_t()
P.wakefield_plot(wake, figsize=(12, 4))

---
## Comparing Models: Accurate vs Fast

`ResistiveWallWakefield` uses FFT-based impedance integration (accurate).
`ResistiveWallPseudomode` uses a damped sinusoid fit (fast, ~10-20% error).

In [ ]:
# Create both models with same parameters
params = dict(
    radius=2.5e-3,
    conductivity=6.5e7,
    relaxation_time=27e-15,
    geometry="round",
)

wake_accurate = ResistiveWallWakefield(**params)
wake_fast = ResistiveWallPseudomode(**params)

print("Accurate model:")
print(f"  {wake_accurate}")
print()
print("Fast model:")
print(f"  {wake_fast}")

In [ ]:
# Compare wake functions
z = np.linspace(-200e-6, 0, 300)

W_accurate = wake_accurate.wake(z)
W_fast = wake_fast.wake(z)

fig, axes = plt.subplots(2, 1, figsize=(10, 6), sharex=True)

ax = axes[0]
ax.plot(-z * 1e6, W_accurate * 1e-12, label="ResistiveWallWakefield (accurate)")
ax.plot(-z * 1e6, W_fast * 1e-12, "--", label="ResistiveWallPseudomode (fast)")
ax.set_ylabel(r"$W(z)$ (V/pC/m)")
ax.legend()
ax.set_title("Wakefield Comparison")

ax = axes[1]
rel_diff = (W_fast - W_accurate) / np.abs(W_accurate).max() * 100
ax.plot(-z * 1e6, rel_diff)
ax.set_xlabel(r"Distance behind source $|z|$ (µm)")
ax.set_ylabel("Relative difference (%)")
ax.set_title("Pseudomode Error")
ax.axhline(0, color="k", lw=0.5)

plt.tight_layout()

### Performance Comparison

In [ ]:
%%timeit -n 10 -r 3
# Accurate model: convolve_density
_ = wake_accurate.convolve_density(density, dz)

In [ ]:
%%timeit -n 100 -r 3
# Fast model: convolve_density
_ = wake_fast.convolve_density(density, dz)

---
## Pseudomode Properties

`ResistiveWallPseudomode` exposes the fitted pseudomode parameters:

In [ ]:
print(f"Characteristic length s₀: {wake_fast.s0*1e6:.2f} µm")
print(f"Dimensionless relaxation Γ: {wake_fast.Gamma:.3f}")
print(f"Resonant wavenumber kᵣ: {wake_fast.kr:.1f} /m")
print(f"Quality factor Qᵣ: {wake_fast.Qr:.2f}")

In [ ]:
# Access the underlying pseudomode
wake_fast.pseudomode

### Bmad Export

`ResistiveWallPseudomode` can export to Bmad format for use in external tracking codes:

In [ ]:
print(wake_fast.to_bmad())

---
## Validation Against SLAC-PUB-10707

Compare our implementation against digitized data from the original paper:

In [ ]:
def validate_against_slacpub(figure_num):
    """Compare against digitized SLAC-PUB-10707 figures."""
    geometry = "round" if figure_num == 4 else "flat"
    radius = 2.5e-3

    # Load digitized data
    data = np.loadtxt(
        f"../data/SLAC-PUB-10707-digitized-Fig{figure_num}-AC-Cu.csv", delimiter=","
    )
    z_ref = data[:, 0] * 1e-6  # µm → m
    # Convert from paper's CGS units
    W_ref = data[:, 1] * 4 / radius**2 / (4 * np.pi * epsilon_0)

    # Create our model
    wake = ResistiveWallWakefield.from_material(
        "copper-slac-pub-10707",
        radius=radius,
        geometry=geometry,
    )

    z = np.linspace(0, 300e-6, 300)
    W = wake.wake(-z)

    plt.figure(figsize=(8, 4))
    plt.plot(z * 1e6, W * 1e-12, label="ResistiveWallWakefield")
    plt.plot(
        z_ref * 1e6, W_ref * 1e-12, "o", ms=4, label=f"SLAC-PUB-10707 Fig. {figure_num}"
    )
    plt.xlabel(r"Distance behind source $|z|$ (µm)")
    plt.ylabel(r"$W(z)$ (V/pC/m)")
    plt.title(f"Validation: {geometry.title()} Copper Pipe")
    plt.legend()


validate_against_slacpub(4)  # Round geometry

In [ ]:
validate_against_slacpub(8)  # Flat geometry

---
## Summary

| Feature | `ResistiveWallWakefield` | `ResistiveWallPseudomode` |
|---------|--------------------------|---------------------------|
| Method | FFT-based impedance | Damped sinusoid fit |
| Accuracy | High | Good (~10-20% error) |
| Speed | Moderate | Very fast |
| Impedance access | ✓ | ✓ (analytical) |
| Bmad export | ✗ | ✓ |
| Best for | General use, validation | Production tracking |

Both classes share the same API:
- `wake(z)` - Evaluate wakefield
- `impedance(k)` - Evaluate impedance
- `convolve_density(density, dz)` - Density convolution
- `particle_kicks(P)` - Per-particle kicks
- `apply_to_particles(P, length)` - Apply to particles
- `plot()` - Visualize wakefield
- `from_material()` - Create from preset